In [23]:
# import numpy as np
import lib2to3.pgen2.token

import pandas as pd
from bs4 import BeautifulSoup
import requests
import os
# import gc

In [2]:
url_a1 = "http://recruit.nchu.edu.tw/college-exam/CAAC/111/111APP_S2StudList/passqual_003.htm"
url_a2 = "https://sxwzdnz8d7lu94mhnzgvag.on.drv.tw/111%E7%94%B3%E8%AB%8B%E5%85%A5%E5%AD%B8%E9%8C%84%E5%8F%96%E5%90%8D%E5%96%AE/020-%E7%B6%B2%E8%B7%AF%E9%8C%84%E5%8F%96%E5%90%8D%E5%96%AE/pass_020_020472.htm"
url_a3 = "http://www.ntcu.edu.tw/newweb/doc/oaa/1110602/passqual_031.htm"
url_a4 = "https://admniu.niu.edu.tw/files/14-1019-34503,r364-1.php?Lang=zh-tw"

url_b1 = "https://recruit.ctu.edu.tw/var/file/44/1044/img/1110627001.html"
url_b2 = "https://rd.asia.edu.tw/uploads/archive_file_multiple/file/62b940418199fb55206131b4/index.html"
url_b3 = "https://www.cyut.edu.tw/~recruit/Interviewstudent/index.html"
url = [url_a1, url_a2, url_a3, url_a4, url_b1, url_b2, url_b3]
# url = [url_a1, url_a2, url_a3, url_a4, url_b2, url_b3] #url_b1怪怪的，先不用


In [3]:
def change_encoding(reponse, encodings: str = 'utf-8'):
    reponse.encoding = encodings
    return reponse


# 一、把取得的html純文字送給BeautifulSoup，產生BeautifulSoup類別

In [4]:
#存到list方便取用做測試
re = [requests.get(i) for i in url]
for i in re:
    i.encoding = "utf-8"
#觀察每個網站的encoding
encoding = [i.encoding for i in re]
#print(encoding)

soup = [BeautifulSoup(i.content, 'html.parser') for i in re]
# re = requests.get(url_a2)
# re.encoding = 'utf-8'
# soup = BeautifulSoup(re.content, 'html.parser')

# 二、找到element
## 透過tag名稱尋找元素(第一個，回傳一個元素類別)

# 中興大學榜單爬蟲處理

In [5]:
elems = soup[0].find_all('a')
#取得所有超連結的連結資訊
# for elem in elems:
#     # elem = elem.text.replace("-", "")
#     # print(elem.text.replace("-",""))
#     print(elem.get('href'))
#     print("----------------------------------")
#url 共同部分
a1_url_same = "http://recruit.nchu.edu.tw/college-exam/CAAC/111/111APP_S2StudList/"

a1_hrefs = [elem.get('href') for elem in elems]  #取得所有超連結的資訊
a1_hrefs = list(set(a1_hrefs))  #去除重複超連結
a1_pages = [a1_url_same + a1_hrefs[i] for i in range(len(a1_hrefs))]  #將連結儲存成一個list
a1_pages.sort()
a1_re = [change_encoding(requests.get(i)) for i in a1_pages]  #將response儲存成一個list
a1_soup = [BeautifulSoup(i.content, 'html.parser') for i in a1_re]  #
a1_spe_pages = [a1_pages[i] for i in range(42, len(a1_pages))]

### 觀察規律，並制定function整理
發現每頁的table總數會有四個，由上而下分別為標題、正取、備取、原住民外加名額

get_names： 取得此頁學系名稱(在後面輸出可以用上)

get_unames： 取得大學名稱(可用在不同的大學url上)

get_dataframe： 要將頁面中的table當中的資料儲存成List供後面轉成dataframe使用， 處理中興大學興翼招生系列特別情況，要加入額外條件，後面的分隔要特別重寫

get_table_names： 取得此頁table的標題們，並轉成list，EX：['招生名額:正取','招生名額:備取']，方便後面整理dataframe的欄位

dataframes： 將每頁的table合成一個list of dataframe，方便後面做合成。 get_dataframe_mss ：處理台中教育大學的音樂系和體育系專長組的取得資料，返回是一個list

get_df_spe： 台中教育大學有兩個學系必須特別寫，music_n，儲存他總共有幾個欄位，EX：[主修:鋼琴_正取,主修:鋼琴_備取]

tables，儲存table的list，最後是用迴圈，將其合成一個df。 get_whole:將dataframes整理成的list of dataframe合成一個dataframe方便輸出，若是台中教育大學的那兩個特殊頁面，就用特別寫的函數去處理，但輸出依樣的是一個dataframe。

In [6]:
#專門拿來爬中興大學榜單的，單位是每一個榜單頁面
class ChungHsing:
    def __init__(self, url_ch):
        """
        :param url_ch:榜單頁裡面的url網址(實際上有表格的)
        :type url_ch: str
        """
        self.url = url_ch
        self.re = change_encoding(requests.get(self.url))
        self.soup = BeautifulSoup(self.re.content, 'html.parser')

    def get_names(self):
        """
        取得此頁url的學系名稱
        """
        name = self.soup.find_all('tr', limit=4)[3].string.strip()
        return name

    def get_uname(self):
        """
        取得大學名稱
        """
        name = self.soup.find_all('b')[0].text
        return name


    def get_dataframe(self, tablechildren, remark):
        """
        這個是要將頁面中的table當裝的資料儲存成List供後面轉成dataframe使用，
        處理中興大學興翼招生系列特別情況，要加入額外條件，後面的分隔要特別重寫
        :param tablechildren: soup.find_all('table')[]某個table的children
        :type tablechildren:
        :param remark:輸出為dataframe時，columns要是甚麼名字
        :type remark:
        :return:
        :rtype:
        """
        li = []
        test_li = []
        for child in tablechildren:
            for tr in child:
                for td in tr:
                    test_li.append(td)
        #檢查她是否是特殊頁面
        if self.url in a1_spe_pages:
            for n in range(len(test_li)):
                if test_li[n] != '\n' and str(test_li[n]) != '<br/>':
                    td = str(test_li[n]).strip().replace(u'\u3000', u' ').replace(u'\xa0', u' ')
                    li.append(td)
        else:
            for tds in test_li:
                if tds != "\n":
                    tds = tds.strip().replace(u'\u3000', u' ').replace(u'\xa0', u' ')
                    li.append(tds)

        li = pd.DataFrame(li, columns=[[f'{self.get_names()}_{remark}']]).reset_index()
        return li


    def get_table_names(self):
        """
        取得此頁table的標題們，並轉成list，EX：['招生名額:正取','招生名額:備取']
        ，方便後面整理dataframe的欄位
        :return:
        :rtype:
        """
        y = self.soup.find_all('b')[4:]
        x1 = []
        x2 = []
        x3 = []
        for i in range(len(y)):
            if y[i].text == "名額類別：招生名額":
                try:
                    g = y[i + 2].text
                    p = True
                except:
                    p = False
                if p:
                    if "正取" in y[i + 1].text:
                        x1.append(f"招生名額:正取")
                    if "備取" in y[i + 2].text:
                        x1.append("招生名額:備取")
                else:
                    if "正取" in y[i + 1].text:
                        x1.append(f"招生名額:正取")
            elif y[i].text == "名額類別：原住民外加名額":
                try:
                    g = y[i + 2].text
                    p = True
                except:
                    p = False
                if p:
                    if "正取" in y[i + 1].text:
                        x2.append("原住民外加名額:正取")
                    if "備取" in y[i + 2].text:
                        x2.append("原住民外加名額:備取")
                elif p is False:
                    if "正取" in y[i + 1].text:
                        x2.append("原住民外加名額:正取")
            elif "名額類別：離島外加名額" in y[i].text:
                try:
                    h = y[i + 1].text
                    g = y[i + 2].text
                    k = True
                except:
                    k = False
                if k:
                    if "正取" in y[i + 1].text:
                        x3.append("離島外加名額:正取")
                    if "備取" in y[i + 2].text:
                        x3.append("離島外加名額:備取")
                else:
                    if "正取" in y[i + 1].text:
                        x3.append("離島外加名額:正取")
        x = x1 + x2 + x3
        return x

    def dataframes(self, ):
        """
        將每頁的table合成一個一個list of dataframe，方便後面做合成。
        :return:
        :rtype:
        """
        d = []
        for i in range(len(self.soup.find_all('table')) - 1):
            # print(i)
            s = self.get_dataframe(tablechildren=self.soup.find_all('table')[i + 1].children,
                                   remark=self.get_table_names()[i])
            d.append(s)
        return d
    @staticmethod
    def get_dataframe_mss(tablechildren):
        """
        音樂系和體育系專長組的取得資料，返回是一個list
        :param tablechildren:
        :type tablechildren:
        :return:
        :rtype:
        """
        t_li = []
        li1 = []
        for child in tablechildren:
            for tr in child:
                for td in tr:
                    t_li.append(td)
        for td1 in t_li:
            if td1 != '\n':
                print(str(td1))
                td1 = td1.text.strip().replace(u'\u3000', u' ').replace(u'\xa0', u' ')
                li1.append(td1)
        return li1

    #直接定義一個新的函數，抓有分樂器主修的
    def get_df_spe(self):
        """
        台中教育大學有兩個學系必須特別寫，
        music_n，儲存他總共有幾個欄位，EX：[主修:鋼琴_正取,主修:鋼琴_備取]
        tables，儲存table的list
        最後是用迴圈，將其合成一個df。
        :return:
        :rtype:
        """
        aa = False
        music_c = self.soup.find_all('b')[4:]
        music_n = []#儲存有幾個欄位的list，方便欄位名
        tables = []#儲存table的list
        main_name = ''
        for i in range(len(music_c)):
            if "主修" in music_c[i].text:
                main_name = music_c[i].text.replace("\u3000", "")  #取得主修名稱
                # print(f'主修:{main_name}')
                try:
                    y = music_c[i + 2]
                    if "備取" in music_c[i + 2].text:
                        aa = True
                    else:
                        aa = False
                except:
                    aa = False
            elif "運動" in music_c[i].text:
                main_name = music_c[i].text.replace("\u3000", "")  #主修名稱
                try:
                    y = music_c[i + 2].text
                    if "備取" in music_c[i + 2].text:
                        aa = True
                except:
                    aa = False
            if aa is True:
                if i+2<len(music_c):
                    if "正取" in music_c[i + 1].text and "0名" not in music_c[i + 1].text:
                        music_n.append(f'{main_name}_正取')
                    if "備取" in music_c[i + 2].text and "0名" not in music_c[i + 2].text:
                        music_n.append(f'{main_name}_備取')
            elif aa is False:
                if len(music_c) - 1 == i:
                    pass
                else:
                    if "正取" in music_c[i + 1].text and "0名" not in music_c[i + 1].text:
                        music_n.append(f'{main_name}_正取')

        for g in range(len(music_n)):
            sss = self.get_dataframe_mss(tablechildren=self.soup.find_all('table')[g + 1].children)
            ss = pd.DataFrame(sss, columns=[[f'{music_n[g]}']]).reset_index()
            tables.append(ss)
        ft = pd.DataFrame()
        for z in range(len(tables) - 1):
            if z == 0:
                ft = pd.merge(tables[z], tables[z + 1], how='outer')
            else:
                ft = pd.merge(ft, tables[z], how='outer')
        return ft

    def get_whole(self):
        """
        台中教育大學的音樂系以及體育學系運動專長組要使用不同的function，
        他們在上面的finction就已經完成dataframe的整合，
        而一般的處理是self.dataframes會將此頁面的所有table整理成list of dataframe
        :return:
        :rtype:
        """
        if "臺中教育大學" in self.get_uname():
            chedu = True
        else:
            chedu = False
        if "音樂" in self.get_names():
            music = True
        else:
            music = False
        if "體育學系運動專長組" in self.get_names():
            sport = True
        else:
            sport = False

        if chedu is True and music is True:
            final = self.get_df_spe()
        elif chedu is True and sport is True:
            final = self.get_df_spe()
        else:
            # a = []
            # for i in range(len(self.dataframes())):
            #     a.append(self.dataframes()[i])
            final = pd.DataFrame()
            for i in range(len(self.dataframes())):
                if i == 0 and len(self.dataframes()) != 1:
                    final = pd.merge(self.dataframes()[i], self.dataframes()[i + 1], how="outer")
                elif i == 0 and len(self.dataframes()) == 1:
                    final = self.dataframes()[i]
                else:
                    final = pd.merge(final, self.dataframes()[i], how='outer')
        return final

In [14]:

#總共有六個較為特別，要特別處理
for i in range(len(a1_pages)):
    m = ChungHsing(a1_pages[i])
    m.get_whole().to_csv(path_or_buf=f'中興大學\\{m.get_uname()}_{m.get_names()}.csv')
    print(f'/中興大學/{m.get_uname()}_{m.get_names()}')


/中興大學/國立中興大學_003012 - 中國文學系
/中興大學/國立中興大學_003022 - 外國語文學系
/中興大學/國立中興大學_003032 - 歷史學系
/中興大學/國立中興大學_003042 - 台灣人文創新學士學位學程
/中興大學/國立中興大學_003052 - 財務金融學系
/中興大學/國立中興大學_003062 - 企業管理學系
/中興大學/國立中興大學_003072 - 資訊管理學系
/中興大學/國立中興大學_003082 - 會計學系
/中興大學/國立中興大學_003092 - 行銷學系
/中興大學/國立中興大學_003102 - 應用經濟學系
/中興大學/國立中興大學_003112 - 法律學系
/中興大學/國立中興大學_003122 - 化學系
/中興大學/國立中興大學_003132 - 物理學系一般物理組
/中興大學/國立中興大學_003142 - 物理學系光電物理組
/中興大學/國立中興大學_003152 - 應用數學系應用數學組
/中興大學/國立中興大學_003162 - 應用數學系數據科學與計算組
/中興大學/國立中興大學_003172 - 機械工程學系
/中興大學/國立中興大學_003182 - 土木工程學系
/中興大學/國立中興大學_003192 - 環境工程學系
/中興大學/國立中興大學_003202 - 化學工程學系
/中興大學/國立中興大學_003212 - 材料科學與工程學系
/中興大學/國立中興大學_003222 - 智慧創意工程學士學位學程
/中興大學/國立中興大學_003232 - 電機工程學系
/中興大學/國立中興大學_003242 - 資訊工程學系
/中興大學/國立中興大學_003252 - 資訊工程學系(APCS組)
/中興大學/國立中興大學_003262 - 電機資訊學院學士班
/中興大學/國立中興大學_003272 - 生物產業機電工程學系
/中興大學/國立中興大學_003282 - 水土保持學系
/中興大學/國立中興大學_003292 - 食品暨應用生物科技學系
/中興大學/國立中興大學_003302 - 農藝學系
/中興大學/國立中興大學_003312 - 園藝學系
/中興大學/國立中興大學_003322 - 森林學系林學組
/中興大學/國立中興大學_003332 - 森林學系木材科學組
/中興大

# 輔仁大學榜單爬蟲

輔仁大學榜單爬蟲
經過觀察網頁架構，取得table name的函式可以共用，因此我從上面的class繼承下去\

get_table:取的輔仁大學企管系綁般的list of dataframe，之所以沒和ChungHsing用同一個，是因為發現她們資料不再同一層，放進去會只拿到個別數字，因此額外再寫一個類似的function dataframesfujen：也是因為當中會用到get_table，要再另外寫一個。

In [17]:
class FuJen(ChungHsing):
    def __init__(self, url_ch):
        super().__init__(url_ch)

    def get_table(self, tablechildren, remark):
        li = []
        for child in tablechildren:
            for tr in child:
                if tr!='\n':
                    ts=str(tr).strip().replace(u'\u3000',' ').replace(u'\xa0',u' ')
                    li.append(ts)

        li = pd.DataFrame(li, columns=[[f'{self.get_names()}_{remark}']]).reset_index()
        return li

    def dataframesfujen(self, ):
        d = []
        for i in range(len(self.soup.find_all('table')) - 1):
            s = self.get_table(tablechildren=self.soup.find_all("table")[i + 1].children,
                                    remark=self.get_table_names()[i])
            d.append(s)
        return d

    def get_whole_fjh(self):
        a = []
        for i in range(len(self.dataframesfujen())):
            a.append(self.dataframesfujen()[i])
        final = pd.DataFrame()
        for i in range(len(a)):
            if i == 0 and len(a) != 1:
                final = pd.merge(a[i], a[i + 1], how="outer")
            elif i == 0 and len(a) == 1:
                final = a[i]
            else:
                final = pd.merge(final, a[i], how='outer')
        return final

In [18]:
fujen = FuJen(url_ch=url_a2)
fujen.get_whole_fjh().to_csv(f'輔仁大學\\輔仁大學_{fujen.get_names()}.csv')

In [57]:
fujen.get_whole_fjh()

,index,020472 - 企業管理學系_招生名額:正取,020472 - 企業管理學系_招生名額:備取,020472 - 企業管理學系_原住民外加名額:正取,020472 - 企業管理學系_原住民外加名額:備取,020472 - 企業管理學系_離島外加名額:正取,020472 - 企業管理學系_離島外加名額:備取
0,0,10095727,10043721,10101223,10317130,10322022,10321927
1,1,10202636,10254425,10123235,10103804,NaN,10322009
2,2,10146216,10064619,10314406,10066827,NaN,10322201
3,3,10031330,10056016,NaN,10164114,NaN,10321933
4,4,10137403,10068129,NaN,10121313,NaN,10322014
...,...,...,...,...,...,...,...
231,231,NaN,10090520,NaN,NaN,NaN,NaN
232,232,NaN,10074617,NaN,NaN,NaN,NaN
233,233,NaN,10201517,NaN,NaN,NaN,NaN
234,234,NaN,10026717,NaN,NaN,NaN,NaN


# 台中教育大學爬蟲
發現跟中興大學的格式依樣，能直接用之前寫的中興大學的class去爬
但有一個學系比較特別 是音樂系跟體育系的，要另外寫規則

In [19]:
url_a3_same = "http://www.ntcu.edu.tw/newweb/doc/oaa/1110602/"

In [20]:
elms1 = soup[2].find_all('a')
a3_hrefs = [elm.get('href') for elm in elms1]
a3_hrefs = a3_hrefs[2:]
a3_hrefs = list(set(a3_hrefs))
a3_hrefs.sort()
a3_pages = [url_a3_same + a3_hrefs[i] for i in range(len(a3_hrefs))]
a3_re = [change_encoding(reponse=requests.get(i)) for i in a3_pages]
a3_soup = [BeautifulSoup(i.content, 'html.parser') for i in a3_re]
a3_pages.__len__()

19

In [24]:
pdf_dir="臺中教育大學/"
if not os.path.exists(pdf_dir):
    os.mkdir(pdf_dir)

for i in range(len(a3_pages)):
    m = ChungHsing(a3_pages[i])
    m.get_whole().to_csv(f'臺中教育大學/{m.get_uname()}_{m.get_names()}.csv')
    print(f'{m.get_uname()}_{m.get_names()}')

國立臺中教育大學_031012 - 教育學系
國立臺中教育大學_031022 - 特殊教育學系
國立臺中教育大學_031032 - 幼兒教育學系
國立臺中教育大學_031042 - 體育學系非運動專長組
國立臺中教育大學_031052 - 語文教育學系
國立臺中教育大學_031062 - 區域與社會發展學系
國立臺中教育大學_031072 - 諮商與應用心理學系
國立臺中教育大學_031082 - 臺灣語文學系
國立臺中教育大學_031092 - 英語學系
國立臺中教育大學_031102 - 英語學系(公費生)
國立臺中教育大學_031112 - 數學教育學系
國立臺中教育大學_031122 - 科學教育與應用學系
國立臺中教育大學_031132 - 資訊工程學系
國立臺中教育大學_031142 - 數位內容科技學系
國立臺中教育大學_031152 - 國際企業學系
國立臺中教育大學_031162 - 文化創意產業設計與營運學系
　10173610 許〇珩
　10086336 李〇瑄
　10310216 黃〇齊
　10043213 盧〇融
　10238935 賴〇婷
　10142105 呂〇萱
　10236118 劉〇慈
　10164111 董〇宏
　10235930 侯〇璘
　10171815 柯〇菡
　10226434 尤〇瑜
　10129129 王〇勒
　10139536 廖〇凱
　10236304 劉〇勻
　10062307 陳〇瑋
　10289831 黃〇靖
　10290908 黃〇茵
　10121031 黃〇軒
　10216934 黃〇茹
　10171728 謝〇欣
　10087131 張〇馨
　10209229 姚〇新
　10234124 莊〇婷
　10282619 蔡〇靜
　10299607 廖〇禔
　10072412 蔡〇涵
　10171729 劉〇瑋
　10261916 黃〇棠
　10288720 李〇平
　10162805 黃〇涵
　10197005 黃〇涵
　10236130 林〇婕
　10127914 林〇彤
　10226330 丁〇鈞
　10227205 王〇玄
　10153301 林〇辰
　10236114 許〇榤
　10209320 曾〇喆
　10302211 陳〇瑜
　10150410 李〇澐
　10223835 蘇〇欣
　1012

# 宜蘭大學爬蟲

In [26]:
headers= {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
}
a4_re=requests.get(url_a4,headers=headers)
a4_re.encoding='utf-8'
a4_soup=BeautifulSoup(a4_re.content,'html.parser')
a4_same='https://admniu.niu.edu.tw'
pdf_list=[]
pdf_name=[]
for h in a4_soup.find_all('a'):
    if '學系' in h.text:
        pdf_list.append(h.get('href'))
        pdf_name.append(h.text)
pdf_url_list=[a4_same+i for i in pdf_list ]
# pdf_dir="minu_pdfs/"
# if not os.path.exists(pdf_dir):
#     os.mkdir(pdf_dir)
for j in range(len(pdf_url_list)):
    file=requests.get(pdf_url_list[j],headers=headers)
    open(f"宜蘭大學_{pdf_name[j]}.pdf",'wb').write(file.content)

In [31]:
import pdfplumber
# path="E:\\new_Desktop\\python\\intervi\\"
# pdf_path=[path+f"宜蘭大學_{pdf_name[k]}.pdf" for k in range(len(pdf_url_list))]
pdf_path=[f"宜蘭大學_{pdf_name[k]}.pdf" for k in range(len(pdf_url_list)) ]

In [81]:
class PdfCrawler:

    def __init__(self,pdf_file_path):
        text=""
        pdf=pdfplumber.open(pdf_file_path)
        for i in range(len(pdf.pages)):
            text=text+pdf.pages[i].extract_text()

        self.text=text
    def get_df_names(self):
        y=(self.text.replace('\n', '  ')
           .replace('-', '系所--')
           .split('系所--')[1]
           .replace('\u3000','')
           .split("(先由左⾄右，再由上⽽下，依考⽣名次排序)"))
        name_list=[]

        for i in y:
            z=i.split(" ")
            for v in z:
                try:
                    int(v)
                except:
                    if v!='' and "〇" not in v:
                        name_list.append(v)
        name_list=name_list[1:]
        name_list=[i for i in name_list if "※備註" not in i] #把會影響我們取值刪除。
        table_names=[]
#         print(f'name_list={name_list}')
        for i in range(len(name_list)-1):
            if "名額類別：招⽣名額" in name_list[i]:
                if "正取" in name_list[i+1] and "備取" in name_list[i+2]:
                    table_names.append(f'名額類別：招⽣名額_正取')
                    table_names.append(f'名額類別：招⽣名額_備取')
                elif "正取" in name_list[i+1]:
                    table_names.append(f'名額類別：招⽣名額_正取')
            elif "名額類別：原住⺠外加名額" in name_list[i]:
                try:
                    name_list[i+2]
                    if "正取" in name_list[i+1] and "備取" in name_list[i+2]:
                        table_names.append(f'名額類別：原住民外加名額_正取')
                        table_names.append(f'名額類別：原住民外加名額_備取')
                    elif "正取" in name_list[i+1]:
                        table_names.append(f'名額類別：原住民外加名額_正取')
                except:
                    if "正取" in name_list[i+1]:

                        table_names.append(f'名額類別：原住民外加名額_正取')
            elif "名額類別：願景計畫外加名額" in name_list[i]:
                try:
                    name_list[i+2]
                    if "正取" in name_list[i+1] and "備取" in name_list[i+2]:
                        table_names.append(f'名額類別：願景計畫外加名額_正取')
                        table_names.append(f'名額類別：願景計畫外加名額_備取')
                except:
                    if "正取" in name_list[i+1]:
                        table_names.append(f'名額類別：願景計畫外加名額_正取')
        return table_names
    def get_df_list(self):
        pdf_text=self.text.replace('\n', '  ').replace('-', '系所--')
        pdf_text=pdf_text.split('系所--')
        pdf_text=pdf_text[1].replace('\u3000',' ').split("(先由左⾄右，再由上⽽下，依考⽣名次排序)")
        name=pdf_text[0].split(" ")[1]
        pdf_text=pdf_text[1:]
        all_list=[]
        # print(pdf_text)
        for j in range(len(pdf_text)):
            temp=pdf_text[j].split(" ")
            print(temp)
            temp_list=[]
            for z in range(len(temp)):
                if "〇" in temp[z]:
                    temp_list.append(temp[z-1]+temp[z])
            all_list.append(temp_list)
        #已取得全部的data Table，現在要解決的是，columns的名稱。
        return all_list
    def get_name(self):
        pdf_text=self.text.replace('\n', '  ').replace('-', '系所--')
        pdf_text=pdf_text.split('系所--')
        pdf_text=pdf_text[1].replace('\u3000','').split("(先由左⾄右，再由上⽽下，依考⽣名次排序)")
        name=pdf_text[0].split(" ")[1]
        return name

    def get_whole(self):
        df_list=[]
        for i in range(len(self.get_df_list())):
            k=pd.DataFrame(self.get_df_list()[i],columns=[[f'{self.get_df_names()[i]}']]).reset_index()
            df_list.append(k)
        final_dataframe=pd.DataFrame()
        for i in range(len(df_list)):
            if i==0 and len(df_list)!=1:
                final_dataframe=pd.merge(df_list[i],df_list[i+1],how='outer')
            elif i==0 and len(df_list)==1:
                final_dataframe=df_list[i]
            else:
                final_dataframe=pd.merge(final_dataframe,df_list[i],how='outer')
        return final_dataframe

In [82]:
pdf_dir="宜蘭大學/"
if not os.path.exists(pdf_dir):
    os.mkdir(pdf_dir)

for i in range(len(pdf_path)):
    y=PdfCrawler(pdf_file_path=pdf_path[i])
    print(f'{i}國立宜蘭大學_{y.get_name()}')
    y.get_whole().to_csv(f'宜蘭大學/國立宜蘭大學_{y.get_name()}.csv')

0國立宜蘭大學_外國語文學系
['', '', '', '10138024', '徐〇睎', '', '10316830', '陳〇琳', '', '10226513', '李〇慧', '', '', '10187229', '洪〇茹', '', '10309821', '林〇任', '', '10170406', '游〇嫻', '', '', '10135805', '林〇勛', '', '10079829', '⾼〇恩', '', '10132601', '戴〇瑄', '', '', '10017509', '湯〇薇', '', '', '備取：18名']
['', '', '', '10009410', '林〇涵', '', '10083935', '張〇', '', '', '10009228', '孫〇淳', '', '', '10015308', '唐〇慈', '', '10209214', '林〇任', '', '10114328', '周〇妤', '', '', '10031905', '王〇安', '', '10121412', '王〇潔', '', '10099306', '沈〇馨', '', '', '10066625', '蔡〇真', '', '10160031', '陳〇琳', '', '10003015', '胡〇沂', '', '', '10132518', '楊〇筑', '', '10087717', '林〇嫻', '', '10174907', '何〇妤', '', '', '10043515', '柯〇婷', '', '10274507', '張〇翰', '', '10104515', '陳〇安', '', '名額類別：原住⺠外加名額', '', '', '正取：2名']
['', '', '', '10140103', '曾〇恩', '', '10311828', '林〇賢', '', '名額類別：願景計畫外加名額', '', '', '正取：2名']
['', '', '', '10066606', '蔡〇樺', '', '10086001', '李〇姗', '', '', '備取：1名']
['', '', '', '10062729', '沈〇璋']
['', '', '', '10138024', '徐〇睎', '', 

# 建國科技大學爬蟲

難以從規律預設函式直接去找table和系名的連結
因此我決定直接用字串的處理，把他整理成table，網頁架構都是同一層，但弄成字串的話還是可以從中找出規律。

並同時返回list of dataframe和每個dataframe代表的系所名稱。

In [37]:
class Chienkuo:
    def __init__(self, url_chk):
        self.url = url_chk
        self.re = change_encoding(requests.get(self.url))
        self.soup = BeautifulSoup(self.re.content, 'html.parser')

    def get_df_list(self):
        urltext = self.soup.text.replace('\n', '  ').replace('--', '系所--')
        urltext = urltext.split('系所--')
        urltext = urltext[1:]
        total = []
        names = []
        for i in urltext:
            if "一般生正取" in i:
                postive = True
            else:
                postive = False
            if "一般生備取" in i:
                alternate = True
            else:
                alternate = False
            if "原住民正取" in i:
                ab_postive = True
            else:
                ab_postive = False
            if "原住民備取" in i:
                ab_alternate = True
            else:
                ab_alternate = False
            if "離島生正取" in i:
                out_postive = True
            else:
                out_postive = False
            if "離島生備取" in i:
                out_alternate = True
            else:
                out_alternate = False
            # print(f'是否有正取:{postive}')
            # print(f'是否有備取:{alternate}')
            # print(i.split("  ")[0])  #取得系所名稱
            name = i.split("  ")[0]
            all_df = []

            #處理正常正備取
            if postive:
                #print(f'正取_{i.split("一般生正取")[1].split("一般生備取")[0]}')

                postive_name = i.split("一般生正取")[1].split("一般生備取")[0].split(" ")[4:]
                if len(postive_name) > 1:
                    postive_name = i.split("一般生正取")[1].split("一般生備取")[0].split(" ")[4:][:-1]
                else:
                    postive_name = i.split("一般生正取")[1].split("一般生備取")[0].split(" ")[4:]
                #print(postive_name)
                postive_df = []
                for n in postive_name:
                    if len(n) <= 11:
                        postive_df.append(n)
                    elif len(n) > 11 and '\u3000' in n:
                        n1 = n[0:11]
                        postive_df.append(n1)
                    else:
                        n1 = n[0:11]
                        n2 = n[11:]
                        postive_df.append(n1)
                        postive_df.append(n2)
                postive_df = pd.DataFrame(postive_df, columns=[[f'{name}_一般生正取']]).reset_index()  #取得正取的dataframe
                # print(postive_df)
                # print(type(postive_df))
                all_df.append(postive_df)
            if alternate:
                #print(i.split("一般生備取")[1])
                alternate_name = i.split("一般生備取")[1].split('[')
                alternate_name = alternate_name[1:]
                alternate_name[-1] = alternate_name[-1].split("\u3000")[0]
                alternate_df = []
                #分字串的時候發現，朝陽大學的，同時有備取跟原住民備取時會出錯，家條件調整
                for z in alternate_name:
                    if "原住民正取" in z:
                        c_index = alternate_name.index(z)
                        # print(c_index)
                        alternate_name = alternate_name[:c_index + 1]
                        alternate_name[-1] = alternate_name[-1].split(" ")[0]
                for m in alternate_name:
                    elm = m.split("]")[1]
                    alternate_df.append(elm)
                alternate_df = pd.DataFrame(alternate_df, columns=[[f'{name}_一般生備取']]).reset_index()
                # print(alternate_df)
                # print(type(postive_df))
                all_df.append(alternate_df)

            #處理原住民生
            if ab_postive:
                ab_postive_name = i.split("原住民正取")[1].split('原住民備取')[0].split(" ")[4:]
                if len(ab_postive_name) > 1:
                    ab_postive_name = i.split("原住民正取")[1].split('原住民備取')[0].split(" ")[4:][:-1]
                else:
                    ab_postive_name = i.split("原住民正取")[1].split('原住民備取')[0].split(" ")[4:]
                ab_postive_df = []
                for p in ab_postive_name:
                    if len(p) <= 12:
                        ab_postive_df.append(p)
                    else:
                        p1 = p[0:11]
                        p2 = p[11:]
                        ab_postive_df.append(p1)
                        ab_postive_df.append(p2)
                ab_postive_df = pd.DataFrame(ab_postive_df, columns=[[f'{name}_原住民正取']]).reset_index()
                all_df.append(ab_postive_df)
            if ab_alternate:
                ab_alternate_name = i.split("原住民備取")[1].split('[')
                ab_alternate_name = ab_alternate_name[1:]
                ab_alternate_name[-1] = ab_alternate_name[-1].split("\u3000")[0]
                ab_alternate_df = []
                for l in ab_alternate_name:
                    elm = l.split("]")[1]
                    ab_alternate_df.append(elm)
                ab_alternate_df = pd.DataFrame(ab_alternate_df, columns=[[f'{name}_原住民備取']]).reset_index()
                all_df.append(ab_alternate_df)

            #處理離島生
            if out_postive:
                out_postive_name = i.split("離島生正取")[1].split('離島生備取')[0].split(" ")[4:]
                if len(out_postive_name) > 1:
                    out_postive_name = i.split("離島生正取")[1].split('離島生備取')[0].split(" ")[4:][:-1]
                else:
                    out_postive_name = i.split("離島生正取")[1].split('離島生備取')[0].split(" ")[4:]
                out_postive_df = []
                for p in out_postive_name:
                    if len(p) <= 12:
                        out_postive_df.append(p)
                    else:
                        p1 = p[0:11]
                        p2 = p[11:]
                        out_postive_df.append(p1)
                        out_postive_df.append(p2)
                out_postive_df = pd.DataFrame(out_postive_df, columns=[[f'{name}_離島生正取']]).reset_index()
                all_df.append(out_postive_df)
            if out_alternate:
                out_alternate_name = i.split("離島生備取")[1].split('[')[1:][-1].split(" ")[:-1]
                out_alternate_df = []
                print(out_alternate_name)
                for l in out_alternate_name:
                    elm = l.split("]")[1]
                    out_alternate_df.append(elm)

                out_alternate_df = pd.DataFrame(out_alternate_df, columns=[[f'{name}_離島生備取']]).reset_index()
                all_df.append(out_alternate_df)
            wholedf = pd.DataFrame()
            for o in range(len(all_df)):
                if len(all_df) == 0:
                    wholedf = pd.DataFrame(columns=[[f'{name}']])
                elif o == 0 and len(all_df) != 1:
                    wholedf = pd.merge(all_df[o], all_df[o + 1], how='outer')
                elif o == 0 and len(all_df) == 1:
                    wholedf = all_df[o]
                else:
                    wholedf = pd.merge(wholedf, all_df[o], how='outer')
            # if postive and alternate:
            #     wholedf = pd.merge(postive_df, alternate_df, how='outer')
            # elif postive and alternate is False:
            #     wholedf = postive_df
            # else:
            #     wholedf = pd.DataFrame(columns=[[f'{name}']])
            # print("------------------")
            total.append(wholedf)
            names.append(name)
        total_return = [total, names]
        return total_return

In [38]:
C = Chienkuo(url_chk=url_b1).get_df_list()


In [39]:
pdf_dir="建國科技大學/"
if not os.path.exists(pdf_dir):
    os.mkdir(pdf_dir)
for u in range(len(C[0])):
    C_name = C[1][u]
    C[0][u].to_csv(f"建國科技大學/建國科技大學_{C_name}.csv")


# 亞洲大學

和建國科技大學的網站架構相同，直接用建國科技大學的class就可以解決

In [40]:
asia = Chienkuo(url_chk=url_b2).get_df_list()

In [42]:
pdf_dir="亞洲大學/"
if not os.path.exists(pdf_dir):
    os.mkdir(pdf_dir)
for t in range(len(asia[0])):
    A_name = asia[1][t]
    print(f'{t}_{A_name}')
    asia[0][t].to_csv(f'亞洲大學/亞洲大學_{A_name}.csv')


0_健康產業管理學系(商業與管理群)
1_食品營養與保健生技學系食品營養組(食品群)
2_食品營養與保健生技學系食品營養組(餐旅群)
3_食品營養與保健生技學系保健化妝品組(家政群生活應用類)
4_醫學檢驗暨生物技術學系(衛生與護理類)
5_醫學檢驗暨生物技術學系(農業群)
6_心理學系(商業與管理群)
7_心理學系(衛生與護理類)
8_生物資訊與醫學工程學系(電機與電子群資電類)
9_資訊工程學系(電機與電子群資電類)
10_行動商務與多媒體應用學系(商業與管理群)
11_資訊傳播學系(設計群)
12_資訊傳播學系(藝術群影視類)
13_經營管理學系(商業與管理群)
14_休閒與遊憩管理學系(商業與管理群)
15_休閒與遊憩管理學系(餐旅群)
16_休閒與遊憩管理學系(外語群日語類)
17_會計與資訊學系(商業與管理群)
18_外國語文學系(商業與管理群)
19_外國語文學系(外語群英語類)
20_外國語文學系(外語群日語類)
21_社會工作學系(商業與管理群)
22_社會工作學系(家政群幼保類)
23_幼兒教育學系(家政群幼保類)
24_數位媒體設計學系(設計群)
25_視覺傳達設計學系(設計群)
26_創意商品設計學系(設計群)
27_時尚設計學系(設計群)
28_時尚設計學系(家政群生活應用類)
29_室內設計學系(土木與建築群)


# 朝陽科技大學
和建國科技大學的網站架構很像，只需要再額外架構原住民正備取的部分即可

還有離島的正取備取問題

根據我處理字串的方式，需額外處理每個table只有一名學生的狀況。

In [43]:
Cyut = Chienkuo(url_chk=url_b3).get_df_list()
Cyut[0][16]

['備1]77510004陳Ｏ其']


,index,工業工程與管理系(工程與管理類)_一般生正取
0,0,54080015陳Ｏ凱


In [44]:
pdf_dir="朝陽科技大學/"
if not os.path.exists(pdf_dir):
    os.mkdir(pdf_dir)

for t in range(len(Cyut[0])):
    Cy_name = Cyut[1][t]
    print(f'{t}_{Cy_name}')
    Cyut[0][t].to_csv(f'朝陽科技大學/朝陽科技大學_{Cy_name}.csv')

0_財務金融系(商業與管理群)
1_企業管理系(商業與管理群)
2_保險金融管理系(商業與管理群)
3_會計系(商業與管理群)
4_會計系(外語群英語類)
5_休閒事業管理系(商業與管理群)
6_休閒事業管理系(外語群英語類)
7_休閒事業管理系(餐旅群)
8_行銷與流通管理系(商業與管理群)
9_行銷與流通管理系(外語群英語類)
10_行銷與流通管理系(餐旅群)
11_銀髮產業管理系(商業與管理群)
12_銀髮產業管理系(家政群幼保類)
13_營建工程系(機械群)
14_營建工程系(土木與建築群)
15_工業工程與管理系(機械群)
16_工業工程與管理系(工程與管理類)
17_工業工程與管理系(商業與管理群)
18_應用化學系(化工群)
19_環境工程與管理系(機械群)
20_環境工程與管理系(電機與電子群電機類)
21_環境工程與管理系(商業與管理群)
22_建築系建築組(土木與建築群)
23_建築系建築組(設計群)
24_建築系室內設計組(設計群)
25_工業設計系(機械群)
26_工業設計系(電機與電子群資電類)
27_工業設計系(設計群)
28_視覺傳達設計系(設計群)
29_景觀及都市設計系(設計群)
30_景觀及都市設計系(農業群)
31_傳播藝術系(設計群)
32_傳播藝術系(商業與管理群)
33_傳播藝術系(藝術群影視類)
34_應用英語系(外語群英語類)
35_幼兒保育系(商業與管理群)
36_幼兒保育系(家政群幼保類)
37_幼兒保育系(家政群生活應用類)
38_社會工作系(商業與管理群)
39_社會工作系(家政群幼保類)
40_社會工作系(餐旅群)
41_資訊管理系資訊管理組(電機與電子群資電類)
42_資訊管理系資訊管理組(商業與管理群)
43_資訊管理系資訊管理組(資管類)
44_資訊管理系數位多媒體組(設計群)
45_資訊管理系數位多媒體組(資管類)
46_資訊工程系資電工程組(電機與電子群電機類)
47_資訊工程系資電工程組(電機與電子群資電類)
48_資訊工程系人工智慧組(機械群)
49_資訊工程系人工智慧組(電機與電子群資電類)
50_資訊工程系人工智慧組(商業與管理群)
51_資訊與通訊系(電機與電子群電機類)
52_資訊與通訊系(電機與電子群資電類)
53_航空機械系(機械群)
54_航空機械系(動力機械群)
55_飛行與民航

# Q3 PDF 爬蟲

In [47]:
pdf1_path="1 萬能科大.pdf"
pdf2_path="2 淡江大學.pdf"
pdf3_path="3 義守大學.pdf"

# 遭遇問題

遇到一個問題，在我的爬蟲條件中，一般生正取、一般生正取這些關鍵字是很重要的

但我直接打字的一般生正取，跟我從pdf的text中複製過來的一般生正取，似乎編碼不同，因此我還要重新去將其貼上才能處理

但是處理成table的方式大致相同。

In [45]:
class PdfCrawlerForQ3:


    def __init__(self,pdf_file_path):
        text=""
        pdf=pdfplumber.open(pdf_file_path)
        for i in range(len(pdf.pages)):
            text=text+pdf.pages[i].extract_text()

        self.text=text
    @staticmethod
    def delete_white(li):
        """
        li為list 來去除當中空白的東西。
        """
        df=[]
        for h in range(len(li)):
            if "Ｏ" in li[h]:
                df.append(li[h])
        return df


    def get_df_list(self):
        pdf_text=self.text.replace('\n', '  ').replace('--', '系所--')
        pdf_text=pdf_text.split('系所--')
        pdf_text = pdf_text[1:]
        total = []
        names = []
        for i in pdf_text:
            if "⼀般⽣正取" in i:
                postive = True
            else:
                postive = False
            if "⼀般⽣備取" in i:
                alternate = True
            else:
                alternate = False
            if "原住⺠正取" in i:
                ab_postive = True
            else:
                ab_postive = False
            if "原住⺠備取" in i:
                ab_alternate = True
            else:
                ab_alternate = False
            if "離島生正取" in i:
                out_postive = True
            else:
                out_postive = False
            if "離島生備取" in i:
                out_alternate = True
            else:
                out_alternate = False
            name=i.split("  ")[0]
            all_df=[]
             #處理正常正備取
            if postive:
                postive_name = i.split("⼀般⽣正取")[1].split("⼀般⽣備取")[0].split(" ")[4:]
                if len(postive_name) > 1:
                    postive_name = i.split("⼀般⽣正取")[1].split("⼀般⽣備取")[0].split(" ")[4:][:-1]
                else:
                    postive_name = i.split("⼀般⽣正取")[1].split("⼀般⽣備取")[0].split(" ")[4:]
                #print(postive_name)
                postive_df=self.delete_white(li=postive_name)
                #不想要有空白格 ，用這個把它去掉
#                 for h in range(len(postive_name)):
#                     if "Ｏ" in postive_name[h]:
#                         postive_df.append(postive_name[h])
                postive_df = pd.DataFrame(postive_df, columns=[[f'{name}_一般生正取']]).reset_index()  #取得正取的dataframe
                # print(postive_df)
                # print(type(postive_df))
                all_df.append(postive_df)
            if alternate:
                #print(i.split("一般生備取")[1])
                alternate_name = i.split("⼀般⽣備取")[1].split('[')
                alternate_name = alternate_name[1:]
                alternate_name[-1] = alternate_name[-1].split("\u3000")[0]
                alternate_df = []
                #分字串的時候發現，朝陽大學的，同時有備取跟原住民備取時會出錯，家條件調整
                for z in alternate_name:
                    if "原住⺠正取" in z:
                        c_index = alternate_name.index(z)
                        # print(c_index)
                        alternate_name = alternate_name[:c_index + 1]
                        alternate_name[-1] = alternate_name[-1].split(" ")[0]
                for m in alternate_name:
                    elm = m.split("]")[1]
                    alternate_df.append(elm)
                alternate_df = pd.DataFrame(alternate_df, columns=[[f'{name}_⼀般⽣備取']]).reset_index()
                # print(alternate_df)
                # print(type(postive_df))
                all_df.append(alternate_df)
                #處理原住民生
            if ab_postive:
                ab_postive_name = i.split("原住⺠正取")[1].split('原住⺠備取')[0].split(" ")[4:]
                if len(ab_postive_name) > 1:
                    ab_postive_name = i.split("原住⺠正取")[1].split('原住⺠備取')[0].split(" ")[4:][:-1]
                else:
                    ab_postive_name = i.split("原住⺠正取")[1].split('原住⺠備取')[0].split(" ")[4:]
                ab_postive_df = self.delete_white(li=ab_postive_name)
#                 for p in ab_postive_name:
#                     if len(p) <= 12:
#                         ab_postive_df.append(p)
#                     else:
#                         p1 = p[0:11]
#                         p2 = p[11:]
#                         ab_postive_df.append(p1)
#                         ab_postive_df.append(p2)

                ab_postive_df = pd.DataFrame(ab_postive_df, columns=[[f'{name}_原住⺠正取']]).reset_index()
                all_df.append(ab_postive_df)
            if ab_alternate:
                ab_alternate_name = i.split("原住⺠備取")[1].split('[')
                ab_alternate_name = ab_alternate_name[1:]
                ab_alternate_name[-1] = ab_alternate_name[-1].split("\u3000")[0]
                ab_alternate_df = []
                for l in ab_alternate_name:
                    elm = l.split("]")[1]
                    ab_alternate_df.append(elm)
                ab_alternate_df = pd.DataFrame(ab_alternate_df, columns=[[f'{name}_原住⺠備取']]).reset_index()
                all_df.append(ab_alternate_df)
            wholedf = pd.DataFrame()
            for o in range(len(all_df)):
                if len(all_df) == 0:
                    wholedf = pd.DataFrame(columns=[[f'{name}']])
                elif o == 0 and len(all_df) != 1:
                    wholedf = pd.merge(all_df[o], all_df[o + 1], how='outer')
                elif o == 0 and len(all_df) == 1:
                    wholedf = all_df[o]
                else:
                    wholedf = pd.merge(wholedf, all_df[o], how='outer')

            total.append(wholedf)
            names.append(name)
        total_return = [total, names]
        return total_return
#         return all_df



In [48]:
pdf1=PdfCrawlerForQ3(pdf_file_path=pdf1_path).get_df_list()

In [55]:
pdf_dir="萬能科技大學/"
if not os.path.exists(pdf_dir):
    os.mkdir(pdf_dir)
for u in range(len(pdf1[0])):
    name=pdf1[1][u]
    print(f'{u}_{name}')
    pdf1[0][u].to_csv(f'萬能科技大學/萬能科技大學_{name}.csv')

0_室內設計與營建科技系室內設計與管理組(機械群)
1_室內設計與營建科技系室內設計與管理組(⼟⽊與建築群)
2_室內設計與營建科技系室內設計與管理組(設計群)
3_資訊⼯程系電機控制組(電機與電⼦群電機類)
4_資訊⼯程系電機控制組(電機與電⼦群資電類)
5_資訊⼯程系遊戲設計與競技組(電機與電⼦群資電類)
6_資訊⼯程系遊戲設計與競技組(設計群)
7_資訊⼯程系遊戲設計與競技組(商業與管理群)
8_航空光機電系(機械群)
9_航空光機電系(動⼒機械群)
10_航空光機電系(電機與電⼦群電機類)
11_精密機械與⼯業⼯程系(機械群)
12_精密機械與⼯業⼯程系(動⼒機械群)
13_⾞輛⼯程系(動⼒機械群)
14_化妝品應⽤與管理系(家政群幼保類)
15_化妝品應⽤與管理系(家政群⽣活應⽤類)
16_化妝品應⽤與管理系(餐旅群)
17_時尚造型設計系時尚造型設計組(家政群⽣活應⽤類)
18_時尚造型設計系時尚造型設計組(餐旅群)
19_時尚造型設計系時尚造型設計組(藝術群影視類)
20_時尚造型設計系時尚表演藝術組(家政群⽣活應⽤類)
21_時尚造型設計系時尚表演藝術組(餐旅群)
22_時尚造型設計系時尚表演藝術組(藝術群影視類)
23_商業設計系(設計群)
24_商業設計系(商業與管理群)
25_商業設計系(家政群⽣活應⽤類)
26_觀光與休閒事業管理系(商業與管理群)
27_觀光與休閒事業管理系(外語群英語類)
28_觀光與休閒事業管理系(餐旅群)
29_餐飲管理系(餐旅群)
30_旅館管理系(商業與管理群)
31_旅館管理系(外語群英語類)
32_旅館管理系(餐旅群)
33_航空暨運輸服務管理系(商業與管理群)
34_航空暨運輸服務管理系(外語群英語類)
35_航空暨運輸服務管理系(餐旅群)
36_航空暨運輸服務管理系國際服務組(家政群⽣活應⽤類)
37_航空暨運輸服務管理系國際服務組(外語群英語類)
38_航空暨運輸服務管理系國際服務組(外語群⽇語類)
39_⾏銷與流通管理系(動⼒機械群)
40_⾏銷與流通管理系(商業與管理群)
41_⾏銷與流通管理系(餐旅群)
42_⾏銷與流通管理系航空暨運籌管理組(商業與管理群)
43_⾏銷與流通管理系航空暨運籌管理組(外語群英語類)
44_企業管理系(動⼒機械群)
45_企業管理系(商業與管理群)
46_企業管理系(餐

In [56]:
pdf2=PdfCrawlerForQ3(pdf_file_path=pdf2_path).get_df_list()

In [57]:
pdf_dir="淡江大學/"
if not os.path.exists(pdf_dir):
    os.mkdir(pdf_dir)

for u in range(len(pdf2[0])):
    name=pdf2[1][u]
    print(f'{u}_{name}')
    pdf2[0][u].to_csv(f'淡江大學/淡江大學_{name}.csv')

0_資訊傳播學系(設計群)
1_化學⼯程與材料⼯程學系(化⼯群)
2_會計學系(商業與管理群)
3_資訊管理學系(資管類)
4_英文學系(外語群英語類)
5_⻄班牙語文學系(外語群⽇語類)
6_俄國語文學系(外語群⽇語類)
7_電機⼯程學系電機通訊組(電機與電⼦群電機類)
8_化學⼯程與材料⼯程學系(食品群)


In [59]:
class PdfCrawlerForQ33:
    """
    for 第三小提的第三個pdf的calss，因為字串裡面讀取起來的編碼不同，所以用不同的class
    """
    def __init__(self, pdf_file_path):
        text=""
        pdf=pdfplumber.open(pdf_file_path)
        for i in range(len(pdf.pages)):
            text=text+pdf.pages[i].extract_text()

        self.text=text

    def get_df_list(self):
        pdf_text=self.text.replace('\n', '  ').replace('--', '系所--')
        pdf_text=pdf_text.split('系所--')
        pdf_text = pdf_text[1:]
        total = []
        names = []
        for i in pdf_text:
            if "一般生正取" in i:
                postive = True
            else:
                postive = False
            if "一般生備取" in i:
                alternate = True
            else:
                alternate = False
            if "原住民正取" in i:
                ab_postive = True
            else:
                ab_postive = False
            if "原住民備取" in i:
                ab_alternate = True
            else:
                ab_alternate = False
            if "離島生正取" in i:
                out_postive = True
            else:
                out_postive = False
            if "離島生備取" in i:
                out_alternate = True
            else:
                out_alternate = False

            name = i.split("  ")[0]
            all_df = []
            #處理正常正備取
            if postive:
                #print(f'正取_{i.split("一般生正取")[1].split("一般生備取")[0]}')

                postive_name = i.split("一般生正取")[1].split("一般生備取")[0].split("  ")[4:]
                if len(postive_name) > 1:
                    postive_name = i.split("一般生正取")[1].split("一般生備取")[0].split("  ")[4:][:-1]
                else:
                    postive_name = i.split("一般生正取")[1].split("一般生備取")[0].split("  ")[4:]
#                 print(postive_name[:-10])
#                 postive_name=postive_name[:-10]
                postive_df = pd.DataFrame(postive_name, columns=[[f'{name}_一般生正取']]).reset_index()  #取得正取的dataframe
#                 print(postive_df)
                # print(type(postive_df))
                all_df.append(postive_df)
            if alternate:
                #print(i.split("一般生備取")[1])
                alternate_name = i.split("一般生備取")[1].split('[')
                alternate_name = alternate_name[1:]
                alternate_name[-1] = alternate_name[-1].split("  ")[0]
                for strip in range(len(alternate_name)):
                    alternate_name[strip]=alternate_name[strip].rstrip("")
#                 print(alternate_name)
                #分字串的時候發現，朝陽大學的，同時有備取跟原住民備取時會出錯，家條件調整
#                 for z in alternate_name:
#                     if "原住民正取" in z:
#                         c_index = alternate_name.index(z)
#                         # print(c_index)
#                         alternate_name = alternate_name[:c_index + 1]
#                         alternate_name[-1] = alternate_name[-1].split(" ")[0]
                for m in range(len(alternate_name)):
                    alternate_name[m] = alternate_name[m].replace("]"," ")
#                     alternate_df.append(elm)
                alternate_df = pd.DataFrame(alternate_name, columns=[[f'{name}_一般生備取']]).reset_index()
                all_df.append(alternate_df)

            #處理原住民生
            if ab_postive:
                ab_postive_name = i.split("原住民正取")[1].split('原住民備取')[0].split(" ")[4:]
                if len(ab_postive_name) > 1:
                    ab_postive_name = i.split("原住民正取")[1].split('原住民備取')[0].split(" ")[4:][:-1]
                else:
                    ab_postive_name = i.split("原住民正取")[1].split('原住民備取')[0].split(" ")[4:]
                ab_postive_df = []
                for p in ab_postive_name:
                    if len(p) <= 12:
                        ab_postive_df.append(p)
                    else:
                        p1 = p[0:11]
                        p2 = p[11:]
                        ab_postive_df.append(p1)
                        ab_postive_df.append(p2)
                ab_postive_df = pd.DataFrame(ab_postive_df, columns=[[f'{name}_原住民正取']]).reset_index()
                all_df.append(ab_postive_df)
            if ab_alternate:
                ab_alternate_name = i.split("原住民備取")[1].split('[')
                ab_alternate_name = ab_alternate_name[1:]
                ab_alternate_name[-1] = ab_alternate_name[-1].split("\u3000")[0]
                ab_alternate_df = []
                for l in ab_alternate_name:
                    elm = l.split("]")[1]
                    ab_alternate_df.append(elm)
                ab_alternate_df = pd.DataFrame(ab_alternate_df, columns=[[f'{name}_原住民備取']]).reset_index()
                all_df.append(ab_alternate_df)

            #處理離島生
            if out_postive:
                out_postive_name = i.split("離島生正取")[1].split('離島生備取')[0].split(" ")[4:]
                if len(out_postive_name) > 1:
                    out_postive_name = i.split("離島生正取")[1].split('離島生備取')[0].split(" ")[4:][:-1]
                else:
                    out_postive_name = i.split("離島生正取")[1].split('離島生備取')[0].split(" ")[4:]
                out_postive_df = []
                for p in out_postive_name:
                    if len(p) <= 12:
                        out_postive_df.append(p)
                    else:
                        p1 = p[0:11]
                        p2 = p[11:]
                        out_postive_df.append(p1)
                        out_postive_df.append(p2)
                out_postive_df = pd.DataFrame(out_postive_df, columns=[[f'{name}_離島生正取']]).reset_index()
                all_df.append(out_postive_df)
            if out_alternate:
                out_alternate_name = i.split("離島生備取")[1].split('[')[1:][-1].split("  ")[:-1]
                out_alternate_df = []
                print(out_alternate_name)
                for l in out_alternate_name:
                    elm = l.split("]")[1]
                    out_alternate_df.append(elm)

                out_alternate_df = pd.DataFrame(out_alternate_df, columns=[[f'{name}_離島生備取']]).reset_index()
                all_df.append(out_alternate_df)
            wholedf = pd.DataFrame()
            for o in range(len(all_df)):
                if len(all_df) == 0:
                    wholedf = pd.DataFrame(columns=[[f'{name}']])
                elif o == 0 and len(all_df) != 1:
                    wholedf = pd.merge(all_df[o], all_df[o + 1], how='outer')
                elif o == 0 and len(all_df) == 1:
                    wholedf = all_df[o]
                else:
                    wholedf = pd.merge(wholedf, all_df[o], how='outer')

            total.append(wholedf)
            names.append(name)
        total_return = [total, names]
        return total_return

In [60]:
pdf3=PdfCrawlerForQ33(pdf_file_path=pdf3_path).get_df_list()

In [61]:
pdf_dir="義守大學/"
if not os.path.exists(pdf_dir):
    os.mkdir(pdf_dir)
for t in range(len(pdf3[0])):
    name=pdf3[1][t]
    print(f'{t}_{name}')
    pdf3[0][t].to_csv(f'義守大學/義守大學_{name}.csv')

0_機械與自動化工程學系(機械群)
1_化學工程學系(化工群)
2_土木工程學系(土木與建築群)
3_創意商品設計學系(設計群)
4_企業管理學系(商業與管理群)
5_財務金融管理學系(商業與管理群)
6_國際商務學系(商業與管理群)
7_醫務管理學系(商業與管理群)
8_休閒事業管理學系(商業與管理群)
9_應用英語學系(外語群英語類)
10_應用日語學系(外語群日語類)
11_觀光學系(餐旅群)
12_餐旅管理學系(餐旅群)
13_廚藝學系(餐旅群)
14_大眾傳播學系(藝術群影視類)
15_國際觀光餐旅學系(餐旅群)
16_國際觀光餐旅學系(外語群英語類)
